In [1]:
import numpy as np
import xarray as xr
import os
from cdo import *
from cdo import Cdo
cdo = Cdo()

import geopandas as geo
import matplotlib.pyplot as plt
import salem


## 裁剪？

In [2]:
#导入shp
cornwall_shp = geo.read_file('/mnt/d/work/lyn/data/CTYUA_DEC_2024_UK_BFC.shp')
#导入需要裁剪的数据

tm = '/mnt/d/work/lyn/data/tm.nc'
pre = '/mnt/d/work/lyn/data/pre.nc'


input = xr.open_dataset(tm)
input = input.t2m
output = input.salem.roi(shape = cornwall_shp)
outputfile = '/mnt/d/work/lyn/data/tm_clip.nc'
output.to_netcdf(path = outputfile)

input = xr.open_dataset(pre)
input = input.tp
output = input.salem.roi(shape = cornwall_shp)
outputfile = '/mnt/d/work/lyn/data/pre_clip.nc'
output.to_netcdf(path = outputfile)

## 分别计算月和年平均

In [18]:
import xarray as xr
import pandas as pd

# 1️⃣ 读取 NetCDF 数据
ds = xr.open_dataset('/mnt/d/work/lyn/data/pre_clip.nc')  # 替换为你的文件路径

# 确保时间维度名称正确
time_dim = "valid_time" if "valid_time" in ds.dims else "time"

# 2️⃣ 获取降水数据（转换单位 m → mm）
pre = ds["tp"] * 1000  # 变量名是 "tp"，单位从 m 转换为 mm

# 3️⃣ 计算年尺度降水累积值（对整个区域求和）
pre_annual = pre.groupby(f"{time_dim}.year").sum(dim=time_dim, skipna=True).sum(dim=["latitude", "longitude"])

# 4️⃣ 计算季节尺度降水累积值
seasons = {
    "DJF": [12, 1, 2],  # 冬季
    "MAM": [3, 4, 5],   # 春季
    "JJA": [6, 7, 8],   # 夏季
    "SON": [9, 10, 11]  # 秋季
}

# 创建空的 DataFrame 来存储季节累积值
seasonal_data = []

for season, months in seasons.items():
    pre_season = pre.sel({time_dim: pre[time_dim].dt.month.isin(months)})
    pre_season = pre_season.groupby(f"{time_dim}.year").sum(dim=time_dim, skipna=True).sum(dim=["latitude", "longitude"])
    
    # 转换为 Pandas DataFrame 并添加季节信息
    df_season = pre_season.to_dataframe().reset_index()
    df_season["season"] = season
    seasonal_data.append(df_season)

# 合并所有季节数据
df_seasonal = pd.concat(seasonal_data)

# 5️⃣ 保存 CSV 文件
df_annual = pre_annual.to_dataframe().reset_index()

# 保存为 CSV 文件
df_annual.to_csv('/mnt/d/work/lyn/data/pre_annual.csv', index=False)
df_seasonal.to_csv('/mnt/d/work/lyn/data/pre_seasonal.csv', index=False)

print("✅ 处理完成，已保存 CSV 文件！")


✅ 处理完成，已保存 CSV 文件！


In [19]:
import xarray as xr
import pandas as pd

# 1️⃣ 读取 NetCDF 数据
ds = xr.open_dataset('/mnt/d/work/lyn/data/tm_clip.nc')  # 替换为你的文件路径

# 确保时间维度名称正确
time_dim = "valid_time" if "valid_time" in ds.dims else "time"

# 2️⃣ 获取温度数据（转换单位 K → °C）
temp = ds["t2m"] - 273.15  # 变量名是 "t2m"，单位从 K 转换为 °C

# 3️⃣ 计算年尺度温度平均值（对整个区域求平均）
temp_annual = temp.groupby(f"{time_dim}.year").mean(dim=time_dim, skipna=True).mean(dim=["latitude", "longitude"])

# 4️⃣ 计算季节尺度温度平均值
seasons = {
    "DJF": [12, 1, 2],  # 冬季
    "MAM": [3, 4, 5],   # 春季
    "JJA": [6, 7, 8],   # 夏季
    "SON": [9, 10, 11]  # 秋季
}

# 创建空的 DataFrame 来存储季节平均值
seasonal_data = []

for season, months in seasons.items():
    temp_season = temp.sel({time_dim: temp[time_dim].dt.month.isin(months)})
    temp_season = temp_season.groupby(f"{time_dim}.year").mean(dim=time_dim, skipna=True).mean(dim=["latitude", "longitude"])
    
    # 转换为 Pandas DataFrame 并添加季节信息
    df_season = temp_season.to_dataframe().reset_index()
    df_season["season"] = season
    seasonal_data.append(df_season)

# 合并所有季节数据
df_seasonal = pd.concat(seasonal_data)

# 5️⃣ 保存 CSV 文件
df_annual = temp_annual.to_dataframe().reset_index()

# 保存为 CSV 文件
df_annual.to_csv('/mnt/d/work/lyn/data/temp_annual.csv', index=False)
df_seasonal.to_csv('/mnt/d/work/lyn/data/temp_seasonal.csv', index=False)

print("✅ 处理完成，已保存 CSV 文件！")


✅ 处理完成，已保存 CSV 文件！
